In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from lib.Stream import Stream, StreamStarSat
from lib.Graph import Graph, StarSat, BHACore
import networkx as nx
import matplotlib.pyplot as plt
import logging

Faire des minidonnées de test, calculer l'intérieur (pour des propriétés de degrés genre hub-autorité etc.)

Pareil pour le pattern enumeration

In [109]:
s = Stream(lang_left=set("abcd"), lang_right=set("abcd"), _loglevel=logging.INFO)
core_property = StreamStarSat(s, threshold=2)
s.setCoreProperty(core_property)
s.readStream("./tests/ChangingNeighbours-StSa-Copy1.json")
s.T

{'alpha': 0, 'omega': 10}

In [110]:
def intersect(i, j):
    # returns the intersection of two time intervals, or -1 is it is void
    # does not make any asumption obout the order of the intervals, however
    # both i and j *are* intervals (i.e. e >= b, and f >= c)
    # TODO: extend to list of intervals
    b, e = i
    c, f = j
    
    # Disjoint
    if c >= e or f <= b:
        return -1
    # Inclusion
    if (c >= b and f <= e) or \
       (b >= c and e <= f):
            return (max(b,c), min(e,f))
    
    # intersection
    if (c <= b and f >= b and f <= e) or\
       (b <= c and e >= c and e <= f):
        return (max(b,c), min(e,f))
    
    return 0

def reduce(s, x, Z1, Z2):
    v = x[0]
    (b, e) = x[1]
    deg_filter = [i for i in s.degrees[v] if intersect(i[1], x[1]) != -1 ]
    print(deg_filter)
    return False



In [145]:
X1 = [(x["u"], (x["b"], x["e"])) for x in s.E ]
X2 = [(x["v"], (x["b"], x["e"])) for x in s.E ]

X = X1 + X2
print(s.interior(X, X))

({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)})


In [152]:
s.bipatterns(X1, X2)

({'d', 'b', 'a', 'c'}, {'b', 'a'}) ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)}) 0
Candidates [('d', 1), ('c', 1)]
[('u', (1, 5)), ('v', (1, 3)), ('v', (2, 4))] [('y', (2, 4))]
q_x=({'d', 'b', 'a', 'c'}, {'d', 'b', 'a'}) has support set S_x = ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)})
({'d', 'b', 'a', 'c'}, {'d', 'b', 'a'}) ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)}) 1
Candidates [('c', 1)]
[('u', (1, 5)), ('v', (1, 3)), ('v', (2, 4))] [('v', (1, 5)), ('x', (1, 3)), ('y', (2, 4))]
q_x=({'d', 'b', 'a', 'c'}, {'d', 'b', 'a', 'c'}) has support set S_x = ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)})
({'d', 'b', 'a', 'c'}, {'d', 'b', 'a', 'c'}) ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)}) 2
Candidates []
[('u', (1, 5)), ('v', (1, 3)), ('v', (2, 4))] [('v', (1, 5)), ('x', (1, 3)), ('y', (2, 4))]
q_x=({'d', 'b', 'a', 'c'}, {'d', 'b', 'a', 'c'}) has support set S_x = ({('v', 1, 4)}, {('x', 1, 3), ('u', 1, 4), ('y', 2, 4)})
({'d', 'b'